In [1]:
import unicodecsv
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [2]:
from datetime import datetime as dt
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
def parse_int(i):
    if i == '':
        return None
    else:
        return int(i)
    

In [3]:
#enrollments = []
#with open('Data/enrollments.csv','rb') as f:
 #   reader = unicodecsv.DictReader(f)
    #for row in reader:
    #    enrollments.append(row)
  #  enrollments = list(reader)
enrollments = read_csv('Data/enrollments.csv')
print enrollments[0]
daily_engagement = read_csv('Data/daily_engagement.csv')
print daily_engagement[0]
project_submissions = read_csv('Data/project_submissions.csv')
print project_submissions[0]

{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}
{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


In [4]:
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    


In [5]:
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    


In [6]:
def unique_number(lst, key):
    a = set()
    for el in lst:
        a.add(el[key])
    return len(list(a))
        
print len(enrollments), unique_number(enrollments, 'account_key')

print len(daily_engagement), unique_number(daily_engagement, 'acct')

print len(project_submissions), unique_number(project_submissions, 'account_key')

1640 1302
136240 1237
3642 743


In [7]:
for engagement in daily_engagement:
    engagement['account_key'] = engagement['acct']
    del engagement['acct']

In [8]:
engaged = set()
for eng in daily_engagement:
    engaged.add(eng['account_key'])
for enroll in enrollments:
    if enroll['account_key'] not in engaged:
        print enroll
        break

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}


In [9]:
count = 0
for enroll in enrollments:
    if enroll['account_key'] not in engaged and enroll['join_date'] != enroll['cancel_date']:
        print enroll
        count+=1
print count

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}
3


In [10]:
uaccounts = set()
for enrol in enrollments:
    if enrol['is_udacity']:
        uaccounts.add(enrol['account_key'])
print len(uaccounts)

6


In [11]:
def delete_uaccounts(lst, uacc):
    temp = []
    for el in lst:
        if el['account_key'] not in uacc:
            temp.append(el)
    return temp

enrollments = delete_uaccounts(enrollments, uaccounts)
daly_engagement = delete_uaccounts(daily_engagement, uaccounts)
project_submissions = delete_uaccounts(project_submissions, uaccounts)

len(enrollments)

1622

In [12]:
paid_students = {}
for enrol in enrollments:
    days_to_cancel = enrol['days_to_cancel']
    key = enrol['account_key']
    date = enrol['join_date']
    if days_to_cancel == None or days_to_cancel>7:
        if key not in paid_students or paid_students[key]<date:
            paid_students[key] = date
len(paid_students)

995

In [13]:
daily_engagement[0]


{'account_key': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [14]:
paid_engaged = []
for eng in daily_engagement:
    key = eng['account_key']
    if key in paid_students:
        delta = eng['utc_date'] - paid_students[key]
        if delta.days<7 and delta.days>=0:
            paid_engaged.append(eng)
print len(paid_engaged)

6919


In [32]:
import numpy as np
def calc_stats(lst, key):
    totals = []
    for acc, engs in lst.items():
        total = 0
        for eng in engs:
            total+=eng[key]
        totals.append(total)
    print np.mean(totals)
    print np.std(totals)
    print np.min(totals)
    print np.max(totals)
    

In [17]:
from collections import defaultdict
engage_by_accounts = defaultdict(list)
for eng in paid_engaged:
    key = eng['account_key']
    engage_by_accounts[key].append(eng)
print("Minutes")    
calc_stats(engage_by_accounts, 'total_minutes_visited')
print "Lessons"
calc_stats(engage_by_accounts, 'lessons_completed')

#total_minutes_by_accounts = {}
#for acc, engs in engage_by_accounts.items():
#    total = 0
#    for eng in engs:
#        total+=eng['total_minutes_visited']
#    total_minutes_by_accounts[acc] = total
    
#import numpy as np
#values = total_minutes_by_accounts.values()
#print np.mean(values)
#print np.std(values)
#print np.min(values)
#print np.max(values)

Minutes
306.708326753
412.996933409
0.0
3564.7332645
Lessons
1.63618090452
3.00256129983
0
36


In [19]:
for acc, engs in engage_by_accounts.items():
    for eng in engs:
        eng['has_visited'] = eng['num_courses_visited'] > 0
calc_stats(engage_by_accounts, 'has_visited')

2.86733668342
2.25519800292
0
7


In [20]:
print project_submissions[0]

{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


In [25]:
paid_students[0]

KeyError: 0

In [42]:
for eng in paid_engaged:
    eng['has_visited'] = paid_engaged['num_courses_visited']>0

In [43]:
passing_engagement = []
non_passing_engagement = []
passed_accounts = set()
subway_project_lesson_keys = ['746169184', '3176718735']
for sub in project_submissions:
    if sub['lesson_key'] in subway_project_lesson_keys and sub['assigned_rating'] in ['PASSED', 'DISTINCTION']:
        passed_accounts.add(sub['account_key'])
for eng in paid_engaged:
    if eng['account_key'] in passed_accounts:
        passing_engagement.append(eng)
    else:
        non_passing_engagement.append(eng)
print len(passing_engagement)
print len(non_passing_engagement)

4527
2392


In [44]:
def group_data(lst):
    lst_by_accounts = defaultdict(list)
    for el in lst:
        key = el['account_key']
        lst_by_accounts[key].append(el)
    return lst_by_accounts

In [45]:
grouped_passing_eng = group_data(passing_engagement)
grouped_nonpassing_eng = group_data(non_passing_engagement)

In [46]:
print "Minutes"
print "Passing"
print calc_stats(grouped_passing_eng, "total_minutes_visited")
print "Non passing"
print calc_stats(grouped_nonpassing_eng, "total_minutes_visited")
print "Lessons"
print "Passing"
print calc_stats(grouped_passing_eng, "lessons_completed")
print "Non passing"
print calc_stats(grouped_nonpassing_eng, "lessons_completed")
print "Visits"
print "Passing"
print calc_stats(grouped_passing_eng, "has_visited")
print "Non passing"
print calc_stats(grouped_nonpassing_eng, "has_visited")


Minutes
Passing
394.586046484
448.499519327
0.0
3564.7332645
None
Non passing
143.326474267
269.538619011
0.0
1768.52274933
None
Lessons
Passing
2.05255023184
3.14222705558
0
36
None
Non passing
0.862068965517
2.54915994183
0
27
None
Visits
Passing
3.38485316847
2.25882147092
0
7
None
Non passing
1.90517241379
1.90573144136
0
7
None


In [55]:
%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns

def plotHist(lst, key_name, fig_num):
    temp = []
    for el in lst:
        temp.append(el[key_name])
    plt.figure(fig_num)
    plt.hist(temp)
    
plotHist(passing_engagement,'total_minutes_visited',0)
plotHist(non_passing_engagement,'total_minutes_visited',0)
plotHist(passing_engagement,'lessons_completed',1)
plotHist(non_passing_engagement,'lessons_completed',1)
plotHist(passing_engagement,'has_visited',2)
plotHist(non_passing_engagement,'has_visited',2)

ImportError: No module named backends